This notebook reconstructs an image processing workflow recommended by GPT Vision 4 to better prepare a low-quality scan for OCR. The intention of this notebook is to critically evaluate AI-recommendations for image processing as opposed to earlier decisions taken by the DigiKAR team. The packages and operations below were explicitly recommended by GPT Vision 4, which reproduces the visual output that Monika Barget also received when prompting directly in the ChatGPT user interface. For the initial prompting, Monika Barget used a page from the 1740 "Staatskalender" published in the Electorate of Mainz. Instances where the script below diverts from the GPT Vision 4 recommendations are highlighted in the comments.

In [3]:
# Import packages recommended by ChatGPT

import os
import glob
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import subprocess

# Confirm import for users
print("Import successful!")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Define paths
GITHUB_REPO = "https://github.com/YOUR_GITHUB_USERNAME/atr-historical-research.git"  # Replace with your actual repo
DATASET_PATH = "/sample_data_jpg"  # Replace with user_data_jpg when working with own samples
OUTPUT_PATH = "/tmp/processed_images" # Temporary folder for processed images

# Confirm when folder path exists
print("Folder found!")

In [ ]:
# Clone dataset if not already present
if not os.path.exists(DATASET_PATH):
    print("Cloning dataset from GitHub...")
    subprocess.run(["git", "clone", GITHUB_REPO, DATASET_PATH])
else:
    print("Dataset already exists!")

In [ ]:
# Ensure output directory exists
os.makedirs(OUTPUT_PATH, exist_ok=True)

In [ ]:
# Select the first image
first_image = os.path.join(DATASET_PATH, filenames[0])

# Confirm selection for users
print(f"Using first found image: {first_image}")

In [ ]:
# Load the image in grayscale (GPT-Vision-4 recommendation)
image = cv2.imread(first_image, cv2.IMREAD_GRAYSCALE) # This AI-"decision" is problematic!

# Error handling added for BYOD lab workshop
if image is None:
    raise ValueError(f"Could not load image: {first_image}")
else:
    print("Valid image loaded!")

In [ ]:
# Image enhancement workflow as created by GPT-Vision-4
# users are welcome to adjust the values and test alternative results

# The main library used here is OpenCV, which Monika Barget also used in her much shorter and simpler
# script for the DigiKAR project. 

# OpenCV (Open Source Computer Vision Library) is an open source computer vision and
# machine learning software library. We should discuss how many of its modules make sense for OCR image
# preparation.

# Full documentation: https://docs.opencv.org/4.x/index.html

# Step 1: Noise Reduction using Non-Local Means Denoising
denoised = cv2.fastNlMeansDenoising(image, h=30, templateWindowSize=7, searchWindowSize=21)

# Step 2: Adaptive Contrast Enhancement using CLAHE
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
contrast_enhanced = clahe.apply(denoised)

# Step 3: Text-Specific Sharpening to Enhance Borders
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
sharpened = cv2.filter2D(contrast_enhanced, -1, kernel)

# Convert to PIL for saving and visualization
enhanced_image = Image.fromarray(sharpened)

# Save the processed image
output_file = os.path.join(OUTPUT_PATH, "enhanced_image.jpg")
enhanced_image.save(output_file)
print(f"Processed image saved to: {output_file}")

# Display the Original vs Enhanced Image
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(image, cmap='gray')
plt.title("Original Image")

plt.subplot(1,2,2)
plt.imshow(sharpened, cmap='gray')
plt.title("AI-Enhanced Image")

plt.show()